# **Final Project-Analysis of Housing Prices**

## Introduction

 Housing is a large part of American culture, with house ownership being a part of the American ideal for many and a way to determine the overall standing of individuals and the country. It is with this in mind that housing data has been used in politics by all sides to attack their respective opposition. From California being unafforable to the supposed mass exoduses towards red states, these memorable but unsupported one-liners have trinkled down to my ears.

I have taken the opportunity provided by this project to determine how valid some of these claims are. As you will see, the conclusion I have drawn for this broad topic will have particular nuances into which specific claims are true and false.

### Imports

In [56]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u342-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [57]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import types as sparktypes
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

sc = SparkContext.getOrCreate();
spark = SparkSession(sc)

## Datasets
### S&P Case-Shiller Home Price Index
For this project, I chose to use the Case-Shiller index as the measure of house pricing, the information of which is publically available on the website of the Federal Reserve Bank of St. Louis. 

This index is measured on a monthly basis and gives data as early as 1987. Data based on this index is available for the national level as well as 20 major cities. For this project, I have chosen to use the national average data as well as the data for the cities of Los Angeles, San Francisco, New York, Miami, Tampa, and Dallas for a total of 7 files.

### Structure
This data initially comes in the form of multiple csv files with 2 columns: the date(MM-01-YYYY) and the price index. Modications have been to add additional attributes for the purposes of this project.
The final schema is:
*   Date
*   Price Index
*   City  (Null in the case of national data)
*   State (Null in the case of national data)
*   Type  ("City" or "National")


### Initial Dataframes
Dataframes are constructed for each chosen city as well as for the national average. All dataframes, with the exception of Dallas, have data from each month starting from January 1987 (January 2000 in the case of Dallas) to May 2022. There is also a table with all the records based on cities starting in January 2000 onward.




In [58]:
# download needed files
#website directory to relevant files: https://fred.stlouisfed.org/release/tables?rid=199&eid=243552#snid=243553 
#backup files directory: https://drive.google.com/drive/folders/1l24l-iIg3a95RwdrSr_kdX7ku1fLbRsj

#national data
!rm -f inflation_national.csv
!wget -O inflation_national.csv "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CSUSHPISA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1987-01-01"
#Los Angelos, CA
!rm -f inflation_LA.csv
!wget -O inflation_LA.csv "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=LXXRSA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1987-01-01"
#San Francisco, CA
!rm -f inflation_SF.csv
!wget -O inflation_SF.csv "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=SFXRSA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1987-01-01"
#New York, NY
!rm -f inflation_NY.csv
!wget -O inflation_NY.csv "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=NYXRSA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1987-01-01"
#Miama, FL
!rm -f inflation_MI.csv
!wget -O inflation_MI.csv "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=MIXRSA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1987-01-01"
#Tampa, FL
!rm -f inflation_TP.csv
!wget -O inflation_TP.csv "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=TPXRSA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1987-01-01"
#Dallas, TX
!rm -f inflation_DA.csv
!wget -O inflation_DA.csv "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DAXRSA&scale=left&cosd=2000-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=2000-01-01"

--2022-08-25 13:56:26--  https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CSUSHPISA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1987-01-01
Resolving fred.stlouisfed.org (fred.stlouisfed.org)... 104.127.84.218
Connecting to fred.stlouisfed.org (fred.stlouisfed.org)|104.127.84.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9086 (8.9K) [text/csv]
Saving to: ‘inflation_national.csv’

inflation_national. 100%[===================>]   8.87K  --.-KB/s    in 0s      

2022-08-25 13:56:27 (113 MB/s

In [59]:
#format csv files into data frames
national_df = spark.read.options(delimiter=",", header = True).csv("inflation_national.csv")
national_df = (national_df.select(col('DATE').cast('date').alias('date'), col('CSUSHPISA').cast('float').alias('price_index'))
              .withColumn("City", lit(None)).withColumn("State", lit(None)).withColumn("Type", lit("National"))
              )
LA_df = spark.read.options(delimiter=",", header = True).csv("inflation_LA.csv")
LA_df = (LA_df.select(col('DATE').cast('date').alias('date'), col('LXXRSA').cast('float').alias('price_index'))
              .withColumn("City", lit("Los Angeles")).withColumn("State", lit("CA")).withColumn("Type", lit("City"))
              )
SF_df = spark.read.options(delimiter=",", header = True).csv("inflation_SF.csv")
SF_df = (SF_df.select(col('DATE').cast('date').alias('date'), col('SFXRSA').cast('float').alias('price_index'))
              .withColumn("City", lit("San Francisco")).withColumn("State", lit("CA")).withColumn("Type", lit("City"))
              )
NY_df = spark.read.options(delimiter=",", header = True).csv("inflation_NY.csv")
NY_df = (NY_df.select(col('DATE').cast('date').alias('date'), col('NYXRSA').cast('float').alias('price_index'))
              .withColumn("City", lit("New York")).withColumn("State", lit("CA")).withColumn("Type", lit("City")) 
              )
MI_df = spark.read.options(delimiter=",", header = True).csv("inflation_MI.csv")
MI_df = (MI_df.select(col('DATE').cast('date').alias('date'), col('MIXRSA').cast('float').alias('price_index'))
              .withColumn("City", lit("Miami")).withColumn("State", lit("FL")).withColumn("Type", lit("City"))
              )
TP_df = spark.read.options(delimiter=",", header = True).csv("inflation_TP.csv")
TP_df = (TP_df.select(col('DATE').cast('date').alias('date'), col('TPXRSA').cast('float').alias('price_index'))
              .withColumn("City", lit("Tampa")).withColumn("State", lit("FL")).withColumn("Type", lit("City"))
              )
#Dallax, Texas does not have data before 2000, unlike the above dfs
DA_df = spark.read.options(delimiter=",", header = True).csv("inflation_DA.csv")
DA_df = (DA_df.select(col('DATE').cast('date').alias('date'), col('DAXRSA').cast('float').alias('price_index'))
              .withColumn("City", lit("Dallas")).withColumn("State", lit("TX")).withColumn("Type", lit("City")) 
              )
#combine CITY dataframes into one through unions
cities_df = (LA_df.unionByName(SF_df).unionByName(NY_df)
            .unionByName(MI_df).unionByName(TP_df).unionByName(DA_df).filter(col("Date") >= (lit("2000-01-01"))))

## Considerations
There is no convenient method of transforming the Case-Shiller Home Price Index into an understandable currency like American dollars. With that in mind, we will also used the national data to get information such as an average month-to-month change to get an idea of how relatively significant these values are.

## Questions

This project is largely designed to generally inquire into the various political-based claims I have been exposed regarding the cities'/states' housing situation are true.

This is the main question analyzed in the course of this project:

> **How does each selected city compare to the national average?**






## Problem Decomposition

1. How does each city compare to the national average?
  1. Create a table where each record of a city is checked to see its price index is greater than the corresponding national average at the corressponding month and year.
  2. Determine the proportion for each city of its price index being greater than the national average.
  3. Get statistical data(mean, standard deviation, etc.) for the index prices of each city.
  4. Create a table for the difference in the index price of each city and the national data for each corresponding month.
  5. Get statistical data(mean, standard deviation, etc.) for the difference between the national price indices and city price indices overall as calculated in Part 4.
  6. Determine the average difference between the city's price indices and the national price indices for each city.
  7. Create a table showing the month-to-month change in the price_index for the national data.
  8. Get statistical data(mean, standard deviation, etc.) for the month-to-month change in the index prices calculated in Part 7.




In [60]:
#Q1
LA_df.createOrReplaceTempView("LA_df")
SF_df.createOrReplaceTempView("SF_df")
NY_df.createOrReplaceTempView("NY_df")
MI_df.createOrReplaceTempView("MI_df")
TP_df.createOrReplaceTempView("TP_df")
DA_df.createOrReplaceTempView("DA_df")
cities_df.createOrReplaceTempView("cities_df")
national_df.createOrReplaceTempView("national_df")

#Part 1
#<date, city, Is_Greater_Than_National(Boolean)
sql_df = spark.sql("""SELECT cities_df.date, cities_df.city, 
                      cities_df.price_index >= national_df.price_index as Is_Greater_Than_National
                  FROM cities_df, national_df 
                  WHERE cities_df.Date = national_df.Date 
                  """)


sql_df.createOrReplaceTempView("sql_df")
sql_df = spark.sql("""SELECT * 
                  FROM sql_df
                  WHERE sql_df.Is_Greater_Than_National = true
                  """)

#Part 2
sql_df.createOrReplaceTempView("cities_gt_national_true")
sql_df = spark.sql("""SELECT city, count(*)/269 as prop_over_national
                  FROM cities_gt_national_true
                  GROUP BY city
                  """)

print("Proportion of each city having a higher price index than the national data:")
sql_df.show(truncate=False)





#Part 3
print("National Price Index Data:")
national_df.where(national_df["date"] >= "2000-01-01").select("price_index").summary().show()
print("Dallas Price Index Data:")
DA_df.select("price_index").summary().show()
print("Tampa Price Index Data:")
TP_df.select("price_index").where(TP_df["date"] >= "2000-01-01").summary().show()
print("Miama Price Index Data:")
MI_df.select("price_index").where(MI_df["date"] >= "2000-01-01").summary().show()
print("Los Angeles Price Index Data:")
LA_df.select("price_index").where(LA_df["date"] >= "2000-01-01").summary().show()




#Part 4
#<date, city, Difference(city-national)
sql_df = spark.sql("""SELECT cities_df.date, cities_df.city, 
                      cities_df.price_index - national_df.price_index as Index_Difference
                  FROM cities_df, national_df 
                  WHERE cities_df.Date = national_df.Date 
                  """)

sql_df.createOrReplaceTempView("price_Diff_DF")
#Part 5
print("Summary Data for price index difference of the cities and the national data:")
sql_df.select("Index_Difference").summary().show()

#Part 6
sql_df = spark.sql("""SELECT city, mean(Index_Difference) 
                  FROM price_Diff_DF
                  GROUP BY city
                  """)
print("Average price index difference of each city and the national data:")
sql_df.show(truncate=False)


#Part 7
sql_df = spark.sql("""SELECT n1.date, n1.price_index - n2.price_index as monthly_index_change
                  FROM national_df as n1, national_df as n2
                  WHERE n1.date - INTERVAL 1 MONTH = n2.date
                  """)

#Part 8
print("Monthy Change in the National Price Index:")
sql_df.select("monthly_index_change").summary().show()




Proportion of each city having a higher price index than the national data:
+-------------+-------------------+
|city         |prop_over_national |
+-------------+-------------------+
|Los Angeles  |1.0                |
|San Francisco|0.7955390334572491 |
|New York     |0.7620817843866171 |
|Miami        |0.9033457249070632 |
|Tampa        |0.6505576208178439 |
|Dallas       |0.01858736059479554|
+-------------+-------------------+

National Price Index Data:
+-------+------------------+
|summary|       price_index|
+-------+------------------+
|  count|               269|
|   mean|168.54142035161695|
| stddev|40.126690100610254|
|    min|           100.552|
|    25%|           141.667|
|    50%|           164.574|
|    75%|           184.599|
|    max|           304.831|
+-------+------------------+

Dallas Price Index Data:
+-------+-----------------+
|summary|      price_index|
+-------+-----------------+
|  count|              269|
|   mean|143.2063167813099|
| stddev|40.8875258263

## Results
There are many nuances in the data to consider when deciding how each city compares to the national average.

Based solely on the proportion calculations, none of our chosen cities(Los Angeles, San Francisco, New York, Miami, Tampa, and Dallas) can be seen as particularly representative of the national average. All the cities except Dallas were skewed towards having a higher than average home prices since 2000,with Dallas having notably lower than average home prices in that same period.

The home price data for each city graphed against the national average would show visible divergence. On average from 2000 to the present, the chosen cities were overall 16.6 points higher on the monthly index, with Los Angeles being 50.8 points higher and Dallas 25.3 points lower. Regardless, these are all huge numbers considering that the average month-month change for the national average price index has only been 0.57 points. 

A statistical analysis will show that the four most skewed cities shared a minimum price index at around 100 before diverging in various ways through each of the quartiles. This suggests that massive differences from the national average(especially with Miami and Los Angeles) in home prices only started after the year 2000.

 

## Conclusion
From this study, it would appear that all the selected cities, with the exception of Dallas, are doing worse off than the national average to varying degrees. Los Angeles is notably the worst of the six cities by a notable margin. With that said, each city's home prices appear to have been around the national average at the start of the year 2000. This would suggest that a mixtures of events and policies from that time onward were responsible the diverging home prices.

I appreciated the opportunity to analyze home pricing data and independently coming to my own conclusion on unsupported political claims that I have been exposed to over the last few years. That being said, I was unable to address one of the key topics I was interested in: a comparison between red states to blue states in their home pricing. It became apparent midway through the project that the type of data was simply not capable of addressing this topic with the selected cities not being guranteed to be representative of the ideal red/blue state. Given a similar project, I wouldn't mind finding new data to address this topic.